<a href="https://colab.research.google.com/github/psygrammer/fast_and_slow/blob/master/drl/notebooks/dqn/tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9. Deep Q Network and Its Variants